<a href="https://colab.research.google.com/github/cras-lab/ML-examples/blob/main/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fashin_mnist 그림을 활용해 AutoEncoder 실습<BR>
원래 이미지 784=(28,28)를 64로 압축한 다음 다시 복원<BR>
우선 필요한 모듈 임포트

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.losses import MeanSquaredError
import numpy as np
import matplotlib.pyplot as plt
#plt.style.use('seaborn-white')


mnist fashion 데이터를 로드<BR>
fashion 데이터는 x는 이미지를 y는 0~9까지의 레이블(각각 티셔츠/탑
바지/스웨터/드레스.. 등)을 반환하는데, <BR>
여기서는 오토인코더이므로 y 값은 필요없음

In [ ]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

로딩된 데이터를 normalize

In [5]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

각각 개수와 형식을 출력해 본다.

In [ ]:
print( x_train.shape)
print(x_test.shape)

최종 압축할 latent dimension을 설정한다.

In [7]:
latent_dim = 64

AutoEncoder class를 정의하여, 인코딩과 디코딩을 수행하도록 해 본다.

In [8]:
class AutoEncoder(Model):
  def __init__(self, latent_dim):
    super(AutoEncoder, self).__init__()

    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential( [Flatten(), Dense(latent_dim, activation='relu')])
    self.decoder = tf.keras.Sequential( [Dense(784, activation ='sigmoid'), Reshape((28,28))])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

AutoEncoder의 인스턴스를 만든뒤, 최적기는 adam, loss는 MSE로 설정한다.

In [9]:
autoencoder = AutoEncoder(latent_dim)
autoencoder.compile( optimizer='adam', loss=MeanSquaredError())

적합화를 수행한다. 에폭은 10회만으로 한다.

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))

도식화를 위해 텐서 객체를 numpy() 객체로 만든다.<BR>
autoencoder.encoder, autoencoder.decoder는 모두 텐서 객체를 반환 <<BR>
따라서 numpy() 메서드를 사용하여 이 텐서를 NumPy 배열로 변환.<BR>
변환된 NumPy 배열은 이미지나 데이터 포인트를 시각화하는 데 사용될 수 있다.

In [11]:
encoded_imgs = autoencoder(x_test).numpy()
decoded_imgs = autoencoder(encoded_imgs).numpy()

10개 그림만 실제로 도식화

In [ ]:
n= 10
plt.figure(figsize=(20,4))

for i in range(n):
  ax = plt.subplot(2, n , i+1)
  plt.imshow(x_test[i])
  plt.title('Original')
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  ax = plt.subplot(2, n , i+1+n)
  plt.imshow(decoded_imgs[i])
  plt.title('Reconstructed')
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

